# Q learning on Cart Pole

This exercise will focus on solving the reinforcement learning problem for the Cart Pole environment.

## Description
This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson in “Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem”. A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

## Action Space
The action is a ndarray with shape (1,) which can take values {0, 1} indicating the direction of the fixed force the cart is pushed with.

Action:
* 0: Push cart to the left
* 1: Push cart to the right

Note: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it

## Observation Space
The observation is a ndarray with shape (4,) with the values corresponding to the following positions and velocities:

Num

Observation

Min

Max

* 0: Cart Position $\in [-4.8, 4.8]$
* 1: Cart Velocity $\in [-\infty, \infty]$
* 2: Pole Angle $\in [~ -0.418 rad (-24°), ~ 0.418 rad (24°)]$
* 3: Pole Angular Velocity $\in [-\infty,\infty]$

Note: While the ranges above denote the possible values for observation space of each element, it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:

The cart x-position (index 0) can take values between (-4.8, 4.8), but the episode terminates if the cart leaves the (-2.4, 2.4) range.

The pole angle can be observed between (-.418, .418) radians (or ±24°), but the episode terminates if the pole angle is not in the range (-.2095, .2095) (or ±12°)

## Rewards
Since the goal is to keep the pole upright for as long as possible, a reward of +1 for every step taken, including the termination step, is allotted. The threshold for rewards is 475 for v1.

## Starting State
All observations are assigned a uniformly random value in (-0.05, 0.05)

## Episode Termination
The episode terminates if any one of the following occurs:

* Pole Angle is greater than ±12°
* Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
* Episode length is greater than 500 (200 for v0)

1. We'll start by installing the gym libraries needed for simulating the environment

In [1]:
!pip install gym
!pip install gym[classic_control]

2. We'll also proceed to the needed imports

In [1]:
import numpy as np # used for arrays

import gym # pull the environment

import time # to measure execution time

import math # needed for calculations

import matplotlib.pyplot as plt # for visualizing

import pygame # has some effect on the rendering

import matplotlib.animation as animation # for making gifs


3. Let's create a variable `env` using the method described in the [documentation](https://www.gymlibrary.ml/environments/classic_control/cart_pole/).

In [2]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

4. take a look at the action space.

In [3]:
env.action_space

Discrete(2)

5. Take a look at the observation space

In [4]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

6. Reset the environment to take a look at a state

In [5]:
env.reset()
# cart position , cart velocity, pole angle, pole angular velocity

(array([-0.02276522,  0.03296215, -0.04455954,  0.04662376], dtype=float32),
 {})

7. Since the environment is continuous in time, we'll make gifs to be able to visualize what the agent is doing. We'll take a few steps to do this:
    * Reset the environment
    * Define an empty list called `arr`
    * Set a variable `done` with value `False`
    * Set a variable `i` with value `0`
    * Start a while loop that will continue as long as `done` is `False` in the loop do:
        * append the visualization of the current state of the environment to the `arr` list (using `env.render(mode='rgb_array')` it should be a numpy array)
        * take a random action to produce the new observation
        * print the new observation

In [7]:
# returns an initial observation
env.reset()

arr = []
done_count = 0
i = 0
while done_count < 30 :
    # env.action_space.sample() produces either 0 (left) or 1 (right).
    arr.append(env.render())
    observation, reward, done, _, info = env.step(env.action_space.sample())
    i+=1
    if done :
        done_count+=1
    print("step", i, observation, reward, done, info)

step 1 [-0.01759298  0.23302849  0.04547037 -0.2892673 ] 1.0 False {}
step 2 [-0.01293241  0.03728863  0.03968502  0.01740269] 1.0 False {}
step 3 [-0.01218664 -0.15837932  0.04003308  0.32233787] 1.0 False {}
step 4 [-0.01535423 -0.3540478   0.04647983  0.62737197] 1.0 False {}
step 5 [-0.02243518 -0.5497866   0.05902727  0.9343233 ] 1.0 False {}
step 6 [-0.03343092 -0.35550848  0.07771374  0.66075754] 1.0 False {}
step 7 [-0.04054108 -0.16154903  0.09092889  0.39352134] 1.0 False {}
step 8 [-0.04377206  0.03217288  0.09879932  0.13083519] 1.0 False {}
step 9 [-0.04312861  0.22575094  0.10141602 -0.12911612] 1.0 False {}
step 10 [-0.03861359  0.02933345  0.0988337   0.1937615 ] 1.0 False {}
step 11 [-0.03802692 -0.1670532   0.10270893  0.5159137 ] 1.0 False {}
step 12 [-0.04136799 -0.3634601   0.1130272   0.83911484] 1.0 False {}
step 13 [-0.04863719 -0.17004791  0.1298095   0.584007  ] 1.0 False {}
step 14 [-0.05203814  0.02303957  0.14148964  0.33486953] 1.0 False {}
step 15 [-0.051

/opt/anaconda3/envs/rl-env/lib/python3.11/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


8. Use the following command to make a gif out of the `arr` list of renderings

In [8]:
import numpy as np
from PIL import Image

imgs = arr
imgs = [Image.fromarray(img) for img in imgs]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("cart_pole_not_trained.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)

9. Since the observation space is continuous and not discrete, we cannot straightforwardly apply Q-learning. The trick here is to convert this continuous state reinforcement learning problem into a discrete reinforcement learning problem by splitting the range of the different observation metrics into categories. Let's do that.
* What's the legal range of the four different metrics that define a non terminal state?
* For making the discretization function though, do no hesitate to assume larger ranges to avoid any errors

* 0: Cart Position $\in [-4.8, 4.8]$ but valid values are $\in [-2.4, 2.4]$ round it to $[-3, 3]$
* 1: Cart Velocity $\in [-\infty, \infty]$ but based on the observations they should be $\in [-5,5]$
* 2: Pole Angle $\in [~ -0.418 rad (-24°), ~ 0.418 rad (24°)]$ but valid values are $\in [-.2095, .2095]$ let's round it to $[-.3, .3]$
* 3: Pole Angular Velocity $\in [-\infty,\infty]$ but based on the observations they should be $\in [-5,5]$ 

10. We would like to split all of our state variables into 51 categories, set up an `Observation` object that is a list `[51,51,51,51]`

In [9]:
Observation = [51, 51, 51, 51]

11. We need to define a function (we'll call it `get_discrete_state`) that turns a continuous state observation into a discrete state observation. The idea is to have 50 categories with category 0 corresponding to the lowest value, and 50 being the highest value. The input from the function should be a state, and the output should be a tuple of integers between 0 and 50.

In [10]:
# setup a random state, a minimum legal state and maximum legal state
state, info = env.reset()
state_min = np.array([-3,-5,-0.3,-5])
state_max = np.array([3,5,0.3,5])

def get_discrete_state(state):
    var_range = (state_max - state_min)
    discrete_state = (state - state_min) / var_range *50
    return tuple(discrete_state.astype(int))

print(get_discrete_state(state))
print(get_discrete_state(state_min))
print(get_discrete_state(state_max))

(np.int64(25), np.int64(24), np.int64(25), np.int64(25))
(np.int64(0), np.int64(0), np.int64(0), np.int64(0))
(np.int64(50), np.int64(50), np.int64(50), np.int64(50))


12. Create the Q table for the discretized reinforcement learning problem, initialize it with zeros. What is its shape?

In [11]:
q_table = np.zeros(shape=(Observation + [env.action_space.n]))
q_table.shape

(51, 51, 51, 51, 2)

13. Let's now initialize the values we will need for the Q-learning algorithm to work:
* `LEARNING_RATE` = 0.1
* `DISCOUNT` = 0.95
* `EPISODES` = 60000
* `total` = 0
* `total_reward` = 0
* `prior_reward` = 0
* `epsilon` = 0.1

In [12]:
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0
prior_reward = 0
epsilon = 0.1

14. Let's now code the Q-learning algorithm, here are the steps:
* Loop over the number of episodes
    * reset the environment
    * discretize the initial state
    * setup `done=False`
    * setup `episode_reward=0`
    * loop until `done` is `True`
        * setup conditions to implement the $\epsilon-greedy$ policy
        * take a step with the chosen action
        * increment the `episode_reward`
        * discretize the new state
        * if the state is not terminal:
            * update the Q-table the Q-learning update rule
        * replace current discrete state with new discrete state

As sanity check, print the average `episode_reward` calculated over the last 1000 episodes.

In [13]:
episode_reward_list = [] # to calculate average reward across the episodes

# strting the loop
for episode in range(EPISODES+1):
    state, info = env.reset()
    discrete_state = get_discrete_state(state) # discretize the initial state
    done = False # initialize done
    episode_reward = 0 # initialize episode reward

    while not done: # loop until termination of an episode
        if np.random.random() < epsilon: # random action with probability epsilon
            action = env.action_space.sample()
        elif np.max(q_table[discrete_state]) == np.min(q_table[discrete_state]): # if no best action can be found, pick a random action
            action = env.action_space.sample()
        else: # pick greedy action with probability 1-epsilon
            action = np.argmax(q_table[discrete_state])
            
        new_state, reward, done, _, info = env.step(action) #step action to get new states, reward, and the "done" status.

        episode_reward += reward #add the reward

        new_discrete_state = get_discrete_state(new_state) # discretize new state

        if not done: #update q-table
            max_future_q = np.max(q_table[new_discrete_state]) # value of the next best action

            current_q = q_table[discrete_state + (action,)] # value of the current action

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q) # q learning update

            q_table[discrete_state + (action,)] = new_q # set new value in q table

        discrete_state = new_discrete_state # replace current state with new state
    
    episode_reward_list.append(episode_reward) # add the episode reward to the list
    
    if episode % 1000 == 0: # calculate the average reward across the last 1000 episodes and reinitialize the reward list
        print("For episode:", episode, "the reward was:", np.mean(episode_reward_list))
        episode_reward_list = []
        if np.mean(episode_reward_list) > 400:
            break

For episode: 0 the reward was: 60.0
For episode: 1000 the reward was: 28.098
For episode: 2000 the reward was: 30.759
For episode: 3000 the reward was: 34.135
For episode: 4000 the reward was: 38.699
For episode: 5000 the reward was: 42.431
For episode: 6000 the reward was: 48.784


/opt/anaconda3/envs/rl-env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/rl-env/lib/python3.11/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


For episode: 7000 the reward was: 54.87
For episode: 8000 the reward was: 58.06
For episode: 9000 the reward was: 63.654
For episode: 10000 the reward was: 70.68
For episode: 11000 the reward was: 83.143
For episode: 12000 the reward was: 88.052
For episode: 13000 the reward was: 95.935
For episode: 14000 the reward was: 92.65
For episode: 15000 the reward was: 99.459
For episode: 16000 the reward was: 97.041
For episode: 17000 the reward was: 100.747
For episode: 18000 the reward was: 108.496
For episode: 19000 the reward was: 105.037
For episode: 20000 the reward was: 108.05
For episode: 21000 the reward was: 105.993
For episode: 22000 the reward was: 110.378
For episode: 23000 the reward was: 113.177
For episode: 24000 the reward was: 116.374
For episode: 25000 the reward was: 112.859
For episode: 26000 the reward was: 116.942
For episode: 27000 the reward was: 122.548
For episode: 28000 the reward was: 116.273
For episode: 29000 the reward was: 120.953
For episode: 30000 the reward

15. Now that the training is done, it seems as though things turned out really well! Reuse the code to make the animation in order to display the behaviour of the trained agent on the CartPole game.

In [14]:
# returns an initial observation
observation, info = env.reset()

arr = []
done=False
i = 0
while not done:
    discrete_state = get_discrete_state(observation)
    arr.append(env.render())
    action = np.argmax(q_table[discrete_state]) #take greedy action
    observation, reward, done, _, info = env.step(action)
    i+=1

In [15]:
imgs = arr
imgs = [Image.fromarray(img) for img in imgs]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("cart_pole_trained.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)

Congratulations! You have successfully solved a reinforcement learning problem with a continuous state space! The discretization technique is very common when faced with a fairly simple continuous state space like this one! Another approach would be to rely on function approximation in order to map the states and actions to different values in order to pick actions using a function (and this is where Deep Neural Networks may play a significant role!).

NB: Note that discretizing a problem like this to use classic Q-learning is in itself already a function approximation ;)